[Reference](https://itnext.io/how-to-explore-and-visualize-ml-data-for-object-detection-in-images-88e074f46361)

In [1]:
!pip install fiftyone ultralytics renumics-spotlight

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.8/677.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.1/677.1 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
import numpy as np

import fiftyone.zoo as foz


# download 1000 images from the COCO dataset with persons
dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="validation",
    label_types=[
        "detections",
    ],
    classes=["person"],
    max_samples=1000,
    dataset_name="coco-2017-person-1k-validations",
)

Migrating database to v0.23.2


INFO:fiftyone.migrations.runner:Migrating database to v0.23.2


INFO:fiftyone.zoo.datasets:Downloading split 'validation' to '/root/fiftyone/coco-2017/validation' if necessary


INFO:fiftyone.utils.coco:Downloading annotations to '/root/fiftyone/coco-2017/tmp-download/annotations_trainval2017.zip'


 100% |██████|    1.9Gb/1.9Gb [6.2s elapsed, 0s remaining, 339.3Mb/s]       


...

INFO:fiftyone.zoo.datasets:Loading 'coco-2017' split 'validation'


 100% |███████████████| 1000/1000 [17.7s elapsed, 0s remaining, 86.2 samples/s]      


INFO:eta.core.utils: 100% |███████████████| 1000/1000 [17.7s elapsed, 0s remaining, 86.2 samples/s]      


Dataset 'coco-2017-person-1k-validations' created


INFO:fiftyone.zoo.datasets:Dataset 'coco-2017-person-1k-validations' created


In [4]:
def xywh_to_xyxyn(bbox):
    """convert from xywh to xyxyn format"""
    return [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]]


row = []
for i, sample in enumerate(dataset):
    labels = [detection.label for detection in sample.ground_truth.detections]
    bboxs = [
        xywh_to_xyxyn(detection.bounding_box)
        for detection in sample.ground_truth.detections
    ]
    bboxs_persons = [bbox for bbox, label in zip(bboxs, labels) if label == "person"]
    row.append([sample.filepath, labels, bboxs, bboxs_persons])

df = pd.DataFrame(row, columns=["filepath", "categories", "bboxs", "bboxs_persons"])
df["major_category"] = df["categories"].apply(
    lambda x: max(set(x) - set(["person"]), key=x.count)
    if len(set(x)) > 1
    else "only person"
)

In [5]:
from renumics import spotlight
spotlight.show(df)

http://127.0.0.1:36419/


In [6]:
spotlight.show(df, embed=["filepath"])

http://127.0.0.1:37043/


In [7]:
from ultralytics import YOLO
detection_model = YOLO("yolov8n.pt")

100%|██████████| 6.23M/6.23M [00:00<00:00, 78.6MB/s]


In [8]:
detections = []
for filepath in df["filepath"].tolist():
    detection = detection_model(filepath)[0]
    detections.append(
        {
            "yolo_bboxs": [np.array(box.xyxyn.tolist())[0] for box in detection.boxes],
            "yolo_conf_persons": np.mean([
                np.array(box.conf.tolist())[0]
                for box in detection.boxes
                if detection.names[int(box.cls)] == "person"
            ]),
            "yolo_bboxs_persons": [
                np.array(box.xyxyn.tolist())[0]
                for box in detection.boxes
                if detection.names[int(box.cls)] == "person"
            ],
            "yolo_categories": np.array(
                [np.array(detection.names[int(box.cls)]) for box in detection.boxes]
            ),
        }
    )
df_yolo = pd.DataFrame(detections)


image 1/1 /root/fiftyone/coco-2017/validation/data/000000000139.jpg: 448x640 1 person, 5 chairs, 1 potted plant, 2 dining tables, 1 tv, 1 refrigerator, 1 clock, 1 vase, 515.8ms
Speed: 20.7ms preprocess, 515.8ms inference, 31.9ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /root/fiftyone/coco-2017/validation/data/000000000785.jpg: 448x640 1 person, 1 skis, 275.6ms
Speed: 2.5ms preprocess, 275.6ms inference, 2.0ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /root/fiftyone/coco-2017/validation/data/000000000872.jpg: 640x640 2 persons, 422.7ms
Speed: 3.4ms preprocess, 422.7ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /root/fiftyone/coco-2017/validation/data/000000000885.jpg: 448x640 4 persons, 1 tennis racket, 283.0ms
Speed: 2.5ms preprocess, 283.0ms inference, 1.8ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /root/fiftyone/coco-2017/validation/data/000000001000.jpg: 480x640 13 persons, 2 tennis rackets, 285.0

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


image 1/1 /root/fiftyone/coco-2017/validation/data/000000002006.jpg: 480x640 3 persons, 1 bus, 431.9ms
Speed: 10.0ms preprocess, 431.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /root/fiftyone/coco-2017/validation/data/000000002153.jpg: 480x640 5 persons, 2 baseball bats, 528.1ms
Speed: 10.0ms preprocess, 528.1ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /root/fiftyone/coco-2017/validation/data/000000002261.jpg: 448x640 1 person, 403.4ms
Speed: 7.6ms preprocess, 403.4ms inference, 1.6ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /root/fiftyone/coco-2017/validation/data/000000002299.jpg: 416x640 19 persons, 2 ties, 339.6ms
Speed: 3.3ms preprocess, 339.6ms inference, 1.9ms postprocess per image at shape (1, 3, 416, 640)

image 1/1 /root/fiftyone/coco-2017/validation/data/000000002431.jpg: 640x480 1 person, 1 wine glass, 2 cups, 1 knife, 1 spoon, 1 bowl, 1 chair, 1 dining table, 378.5ms
Speed: 2.6ms prepr

In [9]:
df_merged = pd.concat([df, df_yolo], axis=1)
spotlight.show(df_merged, embed=["filepath"])

http://127.0.0.1:45135/
